In [ ]:
18446744071961288304 / 10000

In [1]:


import numpy as np
import time
import pandas as pd


from permutationTest import GreenFloatCuda, GreenIntCuda
from significance_of_mean_cuda import significance_of_mean_cuda


In [2]:
np.random.seed(3)
n = 100
m = n
n_samples = 5
A = np.asarray([np.random.randint(0,n,m,dtype=np.int32) for _ in range(n_samples)])
B = np.asarray([np.random.randint(0,n,n,dtype=np.int32) for _ in range(n_samples)])

In [6]:
%time GreenFloatCuda(A,B,10)

CPU times: user 6.65 ms, sys: 98 µs, total: 6.74 ms
Wall time: 5.55 ms


array([0.66979217, 0.29138066, 0.82909381, 0.93644416, 0.06874465])

In [9]:

%time GreenIntCuda(A,B)

CPU times: user 36.3 ms, sys: 1e+03 ns, total: 36.3 ms
Wall time: 35.5 ms


array([0.73329517, 0.43525231, 0.70439175, 0.85663162, 0.08082237])

In [10]:
%time GreenIntCuda(A,B, batch_size=1)

CPU times: user 55.1 ms, sys: 596 µs, total: 55.7 ms
Wall time: 52.7 ms


array([0.73329517, 0.43525231, 0.70439175, 0.85663162, 0.08082237])

In [ ]:
GreenIntDataPreProcess(A[0][np.newaxis,:],B[0][np.newaxis,:])

In [ ]:
%%time 
SGM = significance_of_mean_cuda(dtype_v=np.uint32,dtype_A=np.float64)
SGM.run(A,B)

In [ ]:
SGM.p_values

In [ ]:
%time x = GreenIntCuda(A,B,return_dperm=True)[0]

In [ ]:
batches = 2 
BATCH_SIZE = np.ceil(A / batches)

In [ ]:
for b in batches:
    A[b*BATCH_SIZE:b*BATCH_SIZE+BATCH_SIZE,:]

In [ ]:
allPval = list()
size = len(A)
sample_n = 2000
for j in range(0, 2, size):
    Abatch_div = A[j:j+sample_n,:]
    Bbatch_div = B[j:j+sample_n,:]
    print(Abatch_div.shape)
    
    

In [ ]:
def batch(Arr, n=1):
    l = Arr.shape[0]
    for ndx in range(0, l, n):
        yield Arr[ndx:min(ndx + n, l),:]



In [ ]:
%%time
p_val_list = list()
for i, (a,b) in enumerate(zip(batch(A,2), batch(B,2))):
    if i ==0:
        p_val_arr = GreenIntCuda(a,b,return_dperm=True)[1]
    else:
        p_val_arr = np.concatenate((p_val_arr, GreenIntCuda(a,b,return_dperm=True)[1]))


In [ ]:
p_val_arr

In [ ]:
GreenIntCuda(A,B,return_dperm=True)[0]

In [ ]:
np.array(p_val_list).flatten() == GreenIntCuda(A,B,return_dperm=True)[0]

In [ ]:

def runTestInBatch(Abatch, Bbatch, sample_n, w):
    allPval = list()
    batchSize = len(Abatch)
    for j in range(0, batchSize, sample_n):
        Abatch_div = Abatch[j:j+sample_n]
        Bbatch_div = Bbatch[j:j+sample_n]
        pExactListNorm = exactTest(Abatch_div, Bbatch_div, w, True)
        allPval.append(pExactListNorm)
    return allPval

In [ ]:
%time x = GreenFloatCuda(A,B, 10, return_dperm=True)[0]

In [ ]:
x

In [ ]:
%time x = GreenFloatCuda(A[2][np.newaxis,:],B[2][np.newaxis,:], 10, return_dperm=True)[0]

In [ ]:
x

In [ ]:
GreenFloatDataPreProcess(A[2][np.newaxis,:],B[2][np.newaxis,:],10)

In [ ]:
GreenFloatDataPreProcess(A[0][np.newaxis,:],B[0][np.newaxis,:],10)

In [ ]:
dperm2 = GreenIntCuda(,return_dperm=True)[1]

In [ ]:
dperm2

In [ ]:
dperm1 == dperm2

In [ ]:
a,b = A[2][np.newaxis,:], B[2][np.newaxis,:]
GreenIntDataPreProcess(a,b)

In [ ]:
A = a
B = b
for i in range(20000):
    A = np.r_[A,a]
    B = np.r_[B,a]

In [ ]:
GreenIntCuda(A,B)

In [ ]:
x.shape

In [ ]:
GreenIntCuda(A,B,return_dperm=True)

In [ ]:
a,b = A[0][np.newaxis,:], B[0][np.newaxis,:]

In [ ]:
GreenIntCuda(a,b, return_dperm=True)

In [ ]:
GreenIntDataPreProcess(a,b)

In [ ]:
str(z).replace("  "," ")

In [ ]:
%%time 
p_val_list = list()
for a,b in zip(A,B):
    pval, pdist = GreenFloatCuda(a,b,100,return_dperm=True)
    p_val_list.append(pval[0])

In [ ]:
p_val_arr = np.array(p_val_list)

In [ ]:
SGM = significance_of_mean_cuda(100, dtype_v=np.uint32,dtype_A=np.float64)
SGM.run(A,B)


In [ ]:
SGM.p_values

In [ ]:
p_val_arr

In [ ]:
NotTNP_Arr = np.load("permutationTestCuda/data/notTNP.npy")
TNPdf = np.load("permutationTestCuda/data/TNPdf.npy")

In [ ]:
0.001165628433227539 * 1000

In [ ]:
k = 2

In [ ]:
6.72 * 3000

In [ ]:
%time pval, pdist = GreenFloatCuda(TNPdf[:4000],NotTNP_Arr[:4000],100,return_dperm=True)

In [ ]:
pval

In [ ]:
%%time 
p_val_list = list()
for A,B in zip(TNPdf,NotTNP_Arr):
    pval, pdist = GreenFloatCuda(TNPdf,NotTNP_Arr,6,return_dperm=True)
    p_val_list.append(pval[0])

In [ ]:
TNPdf.shape

In [ ]:
%%time
SGM = significance_of_mean_cuda(100, dtype_v=np.uint32,dtype_A=np.float64)
SGM.run(TNPdf[:300],NotTNP_Arr[:300])


In [ ]:
pval

In [ ]:
SGM.p_values

In [ ]:
3880 - 450


In [ ]:
np.array(p_val_list)

In [ ]:
np.allclose(SGM.p_values, np.array(p_val_list))

In [ ]:
GreenFloatDataPreProcess(TNPdf[8][np.newaxis,:],NotTNP_Arr[8][np.newaxis,:],100)

In [ ]:
GreenFloatDataPreProcess(TNPdf[8][np.newaxis,:],NotTNP_Arr[8][np.newaxis,:],6)

In [ ]:
TNPdf[8]

In [ ]:
SGM.pdist[11]

In [ ]:
GreenFloatCuda(TNPdf[8], NotTNP_Arr[8],6,return_dperm=True)

In [ ]:
GreenFloatCuda(TNPdf[8], NotTNP_Arr[8],6,return_dperm=True)

In [ ]:
url = "https://static-content.springer.com/esm/art%3A10.1038%2Fnature18003/MediaObjects/41586_2016_BFnature18003_MOESM111_ESM.zip"
!mkdir "data/"
!wget -P "data/" "https://static-content.springer.com/esm/art%3A10.1038%2Fnature18003/MediaObjects/41586_2016_BFnature18003_MOESM111_ESM.zip"
!unzip "./data/41586_2016_BFnature18003_MOESM111_ESM.zip" -d "./data/"

In [ ]:
path = "./data/nature18003-s2/"
T1 = pd.read_excel(path + "CPTAC_BC_SupplementaryTable01.xlsx")
T3 = pd.read_excel(path + "CPTAC_BC_SupplementaryTable03.xlsx")

## Extract important column headings

In [ ]:
paiteint_id = T1.columns[0]
stats_col = T1.columns[5: 5 + 3]

### Patient column  

In [ ]:
paiteint_id

### Cancer phenotype coulmns

In [ ]:
stats_col

### Check viable phenotype status

In [ ]:
all_lab = list()
for r in T1[stats_col].iterrows():
    all_lab += list(r[1].values)

In [ ]:
list(set(all_lab))

### Let's drop patient with "Equivocal" and "Indeterminate" status

In [ ]:
drop_label = ['Indeterminate', 'Equivocal']

In [ ]:
T1.head()

In [ ]:
mask1 = [True if (len(set(r[1].values) & set(drop_label)) == 0) else False for r in T1[stats_col].iterrows()]
T1 = T1[mask1]
T1.reset_index(drop=True, inplace=True)

### Let's divide the dataframe into two dataframes with triple negative (TN) and non-triple negative (NTN)

In [ ]:
mask2 = [all([True if v=='Negative' else False for v in r[1].values] ) for r in T1[stats_col].iterrows()]

T1_TNP = T1[np.array(mask2)]
T1_TNP.reset_index(drop=True, inplace=True)

T1_not_TNP = T1[~np.array(mask2)]
T1_not_TNP.reset_index(drop=True, inplace=True)



### Obtain patient ids for NT and NTN

In [ ]:
TNPpateintIds = T1_TNP[paiteint_id].values
NotTNPpateintIds = T1_not_TNP[paiteint_id].values

### Remove genes (rows) containing NaN for both TN and NTN i.e., make sure all patients have same the gene when comparing.

In [ ]:
def getpatientId(ids):
    patitentList = list()
    for i in ids:
        p_id = T3[T3.columns[[i in c for c in T3.columns]]].columns
        patitentList += list(p_id)
    return patitentList

In [ ]:
only_patientDf = T3[getpatientId(NotTNPpateintIds) + getpatientId(TNPpateintIds)]

In [ ]:
only_patientDf.dropna(inplace=True)

In [ ]:
NotTNP_df =  only_patientDf[getpatientId(NotTNPpateintIds)]
TNP_df =  only_patientDf[getpatientId(TNPpateintIds)]

In [ ]:
NotTNP_Arr, TNP_Arr =NotTNP_df.values, TNP_df.values

In [ ]:
NotTNP_Arr.shape

In [ ]:
TNP_Arr.shape

In [ ]:
NotTNP_df.to_csv("permutationTestCuda/data/notTNPdf")
TNP_df.to_csv("permutationTestCuda/data/TNPdf")

### There are 80 patients without triple negative (NTN) and 26 patients with triple negtive (TN), and with a total of 8051 quantified genes.

# Experiment time

## Let's start with a rather small binsize=6.

In [ ]:
%%time 
p_val_list = list()
for A,B in zip(TNP_Arr,NotTNP_Arr):
    pval, pdist = GreenFloatCuda(A, B,40,return_dperm=True)
    p_val_list.append(pval)

In [ ]:
pval, pdist = GreenFloatCuda(A, B,40,return_dperm=True)

In [ ]:
pdist[0][pdist[0]!=0]

In [ ]:
from significance_of_mean_cuda import significance_of_mean_cuda


In [ ]:
%%time

SGM = significance_of_mean_cuda(20, dtype_v=np.uint32,dtype_A=np.float64)
SGM.run(TNP_Arr, NotTNP_Arr)


In [ ]:
A,B = dpermTNP_Arr[0], NotTNP_Arr[0]

In [ ]:
A,B = A[np.newaxis,:], B[np.newaxis,:]

In [ ]:
pval, pdist = GreenFloatCuda(A, B,20,return_dperm=True)

In [ ]:
pval

In [ ]:
SGM = significance_of_mean_cuda(20, dtype_v=np.uint32,dtype_A=np.float64)
SGM.run( A[,:], B[:,np.newaxis])


In [ ]:
SGM.digitized

In [ ]:
NN = SGM.numerator.ravel()

In [ ]:
NN= NN[NN!=0]

In [ ]:
NN / NN.sum()

In [ ]:
pdist[pdist!=0]

In [ ]:
np.array(P_e_list)

In [ ]:
x = pd.DataFrame(TNP_Arr, columns=list(range(TNP_Arr.shape[1]))).T
y = pd.DataFrame(NotTNP_Arr, columns=list(range(NotTNP_Arr.shape[1]))).T
n_samples = x.shape[1]
outputDf = pd.DataFrame(n_samples * [0], columns=["output"]).T

In [ ]:
%R -i x
%R -i y
%R -i outputDf
%R -i n_samples

In [ ]:
%%R
runFastPerm <- function(x,y,output, n_samples) {
    e <- list(mode="vector",length=n_samples)
        for (i in 1:n_samples) {
      
            mStopDiffMean(x[[i]], y[[i]])
            valX = fastPerm(x[[i]], y[[i]], testStat = diffMean)
            
            print(valX)
            valX <-unlist(valX)

      
            output[[i]] <- as.numeric(valX[1])
      
            }
    

    return(output)


}


In [ ]:
start = time.time()
%R out <- runFastPerm(x,y, outputDf, n_samples)
end = time.time()
print(end - start)

In [ ]:
2738 / 60

In [ ]:
%R -o out

In [ ]:
outT = out.T

In [ ]:
P_fp_list =list(outT.values.ravel())

### Parallelized exact test takes ~3s

In [ ]:
start = time.time()

P_mw_list = MWU(TNP_Arr, NotTNP_Arr)

end = time.time()
print(end - start)

### Mann-Whitney exact test takes ~1s

In [ ]:
start = time.time()

P_t_list = ttests(TNP_Arr, NotTNP_Arr, True)

end = time.time()
print(end - start)

In [ ]:
df_e = pd.DataFrame({"p":P_e_list})
df_mw = pd.DataFrame({"p":P_mw_list})
df_tt = pd.DataFrame({"p":P_t_list})
df_fp = pd.DataFrame({"p":list(np.array(P_fp_list) )})

In [ ]:
df_e = qvalues(df_e, pi0=None)
df_mw = qvalues(df_mw, pi0=None)
df_tt = qvalues(df_tt, pi0=None)
df_fp = qvalues(df_fp, pi0=None)

In [ ]:
df_e["q"][df_e["q"]<0.01].shape[0]


In [ ]:
df_mw["q"][df_mw["q"]<0.01].shape[0]

In [ ]:
df_tt["q"][df_tt["q"]<0.01].shape[0]

In [ ]:
df_fp["q"][df_fp["q"]<0.01].shape[0]